In [1]:
#오디오 전처리를 위한 라이브러리
import librosa
import librosa.display as dsp
from IPython.display import Audio

#데이터 전처리를 위한 라이브러리
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

ROOT_DIR = 'c:/data/AudioMnist/'

In [2]:
#모델의 재현성을 위하여 random seed 고정
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(929)

In [3]:
import pandas as pd
train = pd.read_csv(ROOT_DIR+'train_audioMNIST.csv')
train.head()

,file_name,label
0,c:/data/AudioMnist/23/0_23_44.wav,0
1,c:/data/AudioMnist/60/0_60_14.wav,0
2,c:/data/AudioMnist/50/4_50_29.wav,4
3,c:/data/AudioMnist/03/9_03_17.wav,9
4,c:/data/AudioMnist/04/0_04_31.wav,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  22500 non-null  object
 1   label      22500 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 351.7+ KB


In [5]:
train.file_name[0]

'c:/data/AudioMnist/23/0_23_44.wav'

In [6]:
#소리는 기본적으로 특정 주파수를 가지는 sin함수들의 합
#특정 시간에 주파수 성분이 어떻게 구성되어 있는지 확인
#분석은 주파수 분석 기법을 많이 사용
#음성 데이터 학습을 위해 아날로그 데이터로 되어있는 음성 데이터를 디지털 신호로 변환

#sampling rate의 defult값은 22050Hz인데, 16000Hz으로 설정한 이유는 사람의 목소리는 대부분 16000Hz 안에 포함된다고 함.
data, sample_rate = librosa.load(train.file_name[0], sr=16000)
print('sample_rate:', sample_rate, ', audio shape:',data.shape)
print('length:',data.shape[0]/float(sample_rate), 'secs')

#초당 16000개(16000Hz 주파수)의 샘플을 가지고 있는 데이터라는 의미

sample_rate: 16000 , audio shape: (10228,)
length: 0.63925 secs


In [7]:
test = pd.read_csv(ROOT_DIR+'test_audioMNIST.csv')
test.head()

,file_name,label
0,c:/data/AudioMnist/10/1_10_13.wav,1
1,c:/data/AudioMnist/50/4_50_40.wav,4
2,c:/data/AudioMnist/29/7_29_30.wav,7
3,c:/data/AudioMnist/21/7_21_25.wav,7
4,c:/data/AudioMnist/04/2_04_9.wav,2


In [8]:
train['file_name']

0        c:/data/AudioMnist/23/0_23_44.wav
1        c:/data/AudioMnist/60/0_60_14.wav
2        c:/data/AudioMnist/50/4_50_29.wav
3        c:/data/AudioMnist/03/9_03_17.wav
4        c:/data/AudioMnist/04/0_04_31.wav
                       ...                
22495    c:/data/AudioMnist/25/9_25_27.wav
22496     c:/data/AudioMnist/28/7_28_4.wav
22497     c:/data/AudioMnist/51/6_51_3.wav
22498    c:/data/AudioMnist/03/0_03_26.wav
22499     c:/data/AudioMnist/25/1_25_1.wav
Name: file_name, Length: 22500, dtype: object

In [9]:
def train_dataset():
    dataset = []
    for file in tqdm(train['file_name'], colour='green'):
        if 'wav' in file:
            data, sr = librosa.load(file, sr=16000)
            class_label = int(train[train.file_name == file].label)
            dataset.append([data, class_label])

    print('학습 데이터 생성 완료')
    return pd.DataFrame(dataset, columns=['data', 'label'])

In [10]:
def test_dataset():
    dataset = []
    for file in tqdm(test['file_name'], colour='blue'):
        if 'wav' in file:
            data, sr = librosa.load(file, sr=16000)
            class_label = int(test[test.file_name == file].label)
            dataset.append([data, class_label])

    print('검증 데이터 생성 완료')
    return pd.DataFrame(dataset, columns=['data', 'label'])

In [11]:
train_wav = train_dataset()

100%|██████████| 22500/22500 [33:36<00:00, 11.16it/s]

학습 데이터 생성 완료


In [12]:
test_wav = test_dataset()

100%|██████████| 7500/7500 [11:16<00:00, 11.08it/s]

검증 데이터 생성 완료


In [21]:
train_wav.head()

,data,label
0,"[-2.5602067e-05, 5.113522e-06, -2.7813609e-05,...",0
1,"[-5.5731027e-05, -0.00016563521, -0.0001917473...",0
2,"[-9.397177e-05, -0.00014913721, -0.0001808885,...",4
3,"[1.7803764e-05, 4.432852e-05, 3.4187742e-05, 5...",9
4,"[-0.00015264732, -0.000223465, -0.00018511842,...",0


In [26]:
train_X = np.array(train_wav.data)
test_X = np.array(test_wav.data)

train_y = np.array(train_wav.label)
test_y = np.array(test_wav.label)

#음성은 각각 다른 길이를 갖고 있음.
# 길이가 가장 작은 길이의 데이터를 기준으로 데이터를 잘라서 사용 -> Under Sampling

#가장 작은 길이 구하기
def get_mini(data):
    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)
    return mini

#길이에 맞게 잘라냄
def set_length(data, d_mini):
    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)
    return result

In [35]:
train_X

array([array([-2.5602067e-05,  5.1135221e-06, -2.7813609e-05, ...,
              -3.8511094e-05, -4.4227359e-05,  0.0000000e+00], dtype=float32),
       array([-5.5731027e-05, -1.6563521e-04, -1.9174733e-04, ...,
               6.7143403e-05,  8.0324862e-06,  0.0000000e+00], dtype=float32),
       array([-9.3971772e-05, -1.4913721e-04, -1.8088850e-04, ...,
              -2.7155949e-04, -3.1073933e-04,  0.0000000e+00], dtype=float32),
       ...,
       array([-3.1695387e-04, -4.7771470e-04, -4.5766350e-04, ...,
              -1.0805244e-04, -6.4930202e-05,  0.0000000e+00], dtype=float32),
       array([4.9645707e-05, 9.6644086e-05, 1.1572056e-04, ..., 2.0901030e-05,
              6.0682945e-05, 1.8038925e-05], dtype=float32)                   ,
       array([ 0.00016609,  0.0003012 ,  0.00024623, ..., -0.00029546,
              -0.00035234,  0.        ], dtype=float32)               ],
      dtype=object)

In [36]:
train_y

array([0, 0, 4, ..., 6, 0, 1], dtype=int64)

In [15]:
train_mini = get_mini(train_X)
test_mini = get_mini(test_X)

mini = np.min([train_mini, test_mini])
print('가장 작은 길이:',mini)

가장 작은 길이: 4691


In [16]:
#전처리 완료
train_X = set_length(train_X, mini)
test_X = set_length(test_X, mini)

print('train 전처리 결과:',train_X.shape)
print('test 전처리 결과:',test_X.shape)

train 전처리 결과: (22500, 4691)
test 전처리 결과: (7500, 4691)


In [33]:
#음성 데이터를 load 했으면 이 음성 데이터의 특징을 추출해야 함.
#음성 raw data를 그대로 사용하면 파라미터가 너무 많아지기도 하고 데이터 용량이 너무 커짐.

#퓨리에 변환 -> 
#"안녕하세요"라는 말에서, 어떤 사람은 1초, 어떤 사람은 3초가 걸릴 수도 있음
# 따라서 이 천차만별인 길이에 대하여 같은 "안녕하세요"라는 음성이라고 학습시키기는 어려울 것.
# MFCC (Mel-frequency cepstral coefficients) 알고리즘을 이용
#   음성데이터를 모두 20~40ms 단위로 쪼개고, 쪼갠 단위에 대해서 Mel값을 뽑아서 Feature로 사용.
# Mel-scale : Mel은 사람의 달팽이관을 모티브로 따온 값.
#   주파수 대역중에서 감지가 용이한 주파수를 filer, scaling해줄 수 있음-> 이 기준을 Mel-scale 이라 함.

#예시로, train_X에서 첫번째 음성의 MFCC 특징을 추출
extracted_features = librosa.feature.mfcc(y=train_X[0], sr=16000, n_mfcc=40)
extracted_features.shape
#출력값: (n_mfcc, time_step)

(40, 20)

In [38]:
def preprocess_dataset(data):
    mfccs = []
    for i in tqdm(data, colour='CYAN'):
        extracted_features = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=40)
        extracted_features = np.mean(extracted_features.T, axis=0)
        mfccs.append(extracted_features)
    return mfccs

train_X = preprocess_dataset(train_X)
train_X = np.array(train_X)
print(train_X.shape)

test_X = preprocess_dataset(test_X)
test_X = np.array(test_X)
print(test_X.shape)

100%|██████████| 22500/22500 [03:27<00:00, 108.44it/s]


(22500, 40)


100%|██████████| 7500/7500 [01:11<00:00, 104.63it/s]

(7500, 40)


In [39]:
train_X

array([[-6.2604041e+02,  1.1418032e+02,  1.3868665e+01, ...,
         2.5491018e+00, -3.6942570e+00,  4.7405356e-01],
       [-6.2980090e+02,  8.3690216e+01,  2.8933178e+01, ...,
         8.6487379e+00, -3.5109191e+00, -5.7271767e-01],
       [-6.3775867e+02,  1.3396536e+02,  1.4822125e+01, ...,
         2.9129839e+00, -3.6431846e-01,  4.8412247e+00],
       ...,
       [-5.3629083e+02,  2.4669645e+00,  3.6916222e+01, ...,
         1.1425340e+01,  2.4241509e+00,  2.9575248e+00],
       [-6.3439972e+02,  8.6175827e+01,  2.5325974e+01, ...,
        -1.2430583e+00, -5.4197960e+00,  1.0434834e+00],
       [-6.4664111e+02,  1.2939424e+02,  1.3351482e+01, ...,
        -4.0698695e+00, -2.2186859e+00,  5.1485538e+00]], dtype=float32)

In [40]:
train_y

array([0, 0, 4, ..., 6, 0, 1], dtype=int64)

In [41]:
#RandomForestClassifier 모델
# 예측해야할 0~9 라벨이 분류 변수
# 의사 결정 트리는 feature 별 가지치기를 통해 데이터를 학습하는 알고리즘
from sklearn.ensemble import RandomForestClassifier

#모델 선언
model = RandomForestClassifier()
#모델 학습
model.fit(train_X, train_y)

RandomForestClassifier()

In [44]:
#점수를 메기는 방법인 평가 지표(Metric)를 정의
def ACCURACY(true, pred):
    score = np.mean(true == pred)
    return score

In [45]:
# 모델의 예측과 실제 정답값을 비교.
prediction = model.predict(test_X)
score = ACCURACY(test_y, prediction)
print(f'모델의 정확도는 {score*100:.2f}% 입니다.')

모델의 정확도는 96.83% 입니다.
